In [1]:
from fastai.vision.all import *
import fastai
import timm
import pandas as pd
import numpy as np
from skimage import io as skiio

In [2]:
import torch
torch.cuda.set_device(1)

In [3]:
df = pd.read_csv('../dataset.csv')

In [4]:
df = df.drop(df[df.masa<0].index)

In [5]:
dftrain = df[(df.split=='train') | (df.split=='valid')]
dftest = df[(df.split=='test')]

In [6]:
# !mkdir ../datasetClean/train
# !mkdir ../datasetClean/test

In [7]:
# import shutil
# from imutils import paths

In [8]:
# for image in dftrain.image_id.values:
#     shutil.move(image.replace('datasetClean','../datasetClean'),
#                 image.replace('datasetClean','../datasetClean/train'))

In [9]:
# for image in dftest.image_id.values:
#     shutil.move(image.replace('datasetClean','../datasetClean'),
#                 image.replace('datasetClean','../datasetClean/test'))

In [10]:
# df.volumen.describe()

In [11]:
def get_masa(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['masa'])

def get_volumen(f):
    return float(df[df['image_id']=='datasetClean/'+f.name]['volumen'])

def is_valid(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='valid'

In [12]:
def my_open_file(fn, chnls=None, cls=torch.Tensor):
    im = torch.from_numpy(skiio.imread(str(fn))).type(torch.float32)
    if chnls is not None: im = im[chnls]
    return cls(im)

class MSTensorImage(TensorImage):
    
    def __init__(self, x, chnls_first=False):
        self.chnls_first = chnls_first

    @classmethod
    def create(cls, data:(Path,str,ndarray), chnls=None, chnls_first=True):
        
        if isinstance(data, Path) or isinstance(data, str):
            if str(data).endswith('tif'): 
                im = my_open_file(fn=data, chnls=chnls, cls=torch.Tensor)

        elif isinstance(data, ndarray): 
            im = torch.from_numpy(data)
        else:
            im = data
        
        return cls(im, chnls_first=chnls_first)

    
    def show(self, chnls=[3, 2, 1], bright=1., ctx=None):
        
        if img.ndim > 2:
            visu_img = self[..., chnls] if not self.chnls_first else self.permute([1, 2, 0])[..., chnls]
        else:
            visu_img = self
        
        visu_img = visu_img.squeeze()
        
        visu_img *= bright
        visu_img = np.where(visu_img > 1, 1, visu_img)
        visu_img = np.where(visu_img < 0, 0, visu_img)
        
        plt.imshow(visu_img) if ctx is None else ctx.imshow(visu_img)
        
        return ctx
    
    def __repr__(self):
        
        return (f'MSTensorImage: {self.shape}')

In [13]:
def getDataBlock(presize=224,size=224):
    return DataBlock(blocks = (TransformBlock(type_tfms=partial(MSTensorImage.create, chnls_first=True)), 
                               RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_valid),
                 get_y=get_masa)

In [14]:
path = Path('../datasetStack/')
trainPath = path/'train'

In [15]:
db = getDataBlock()
dls = db.dataloaders(trainPath,bs=128)

In [16]:
from fastai.vision.all import *
callbacks = [
    ShowGraphCallback(),
    #EarlyStoppingCallback(patience=3),
    SaveModelCallback(fname='resnet50Masav2Stack')
]

In [17]:
from fastai.metrics import mae,rmse,mse

In [18]:
learn = cnn_learner(dls,resnet50,loss_func=MSELossFlat(), metrics=[mae,rmse,mse],cbs=callbacks).to_fp16()

In [19]:
# learn.lr_find()

In [20]:
learn.fine_tune(1000,base_lr=3e-2)

epoch,train_loss,valid_loss,mae,_rmse,mse,time


tensor([[[[0.4850]],

         [[0.4560]],

         [[0.4060]]]], device='cuda:1')


RuntimeError: The size of tensor a (5) must match the size of tensor b (3) at non-singleton dimension 1

In [ ]:
# learn.load('resnetrs50Masav2')

In [ ]:
def is_test(f):
    return (df[df['image_id']=='datasetClean/'+f.name]['split']).values[0]=='test'

In [ ]:
dbtest = DataBlock(blocks = (ImageBlock, RegressionBlock()),
                 get_items=get_image_files, 
                 splitter=FuncSplitter(is_test),
                 get_y=get_masa,
                 item_tfms = Resize(512),
                 batch_tfms=[*aug_transforms(size=512,flip_vert=True,
                                            max_zoom=1.0), Normalize.from_stats(*imagenet_stats)])

In [ ]:
dlstest = dbtest.dataloaders(path,bs=16)

In [ ]:
learn.dls=dlstest

In [ ]:
learn.validate()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
preds,gt = learn.tta()

In [ ]:
mean_absolute_error(gt,preds)